In [ ]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "xxxx"
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

os.environ["OPENAI_API_KEY"] = "xxxx"

os.environ["TAVILY_API_KEY"] = "xxxx"

os.environ["COHERE_API_KEY"] = "xxxx"

In [2]:
!pip install langchain-core==0.2.30 langchain-openai==0.1.21 \
    langchain-community==0.2.12 GitPython==3.1.43 \
    langchain-chroma==0.1.2 chromadb==0.5.3 \
    ragas==0.1.14 nest-asyncio==1.6.0 pydantic==2.10.6

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
Reason for being yanked: Regression. AzureChatOpenAI json-mode broken. Fixed in 0.1.22.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 102.3 MB/s eta 0:00:0000:0100:01
  Created wheel for chroma-hnswlib: filename=chroma_hnswlib-0.7.3-cp312-cp312-linux_x86_64.whl size=2279317 sha256=d4bb2c94d55662a943f6a520fa

## ドキュメントの読み込み

In [2]:
from langchain_community.document_loaders import GitLoader


def file_filter(file_path: str) -> bool:
    return file_path.endswith(".md")


loader = GitLoader(
    clone_url="https://github.com/open-mmlab/mmsegmentation",
    repo_path="./mmsegmentation",
    branch="main",
    file_filter=file_filter,
)

documents = loader.load()
print(len(documents))

172


## Ragas による合成テストデータ生成

In [13]:
for document in documents:
    document.metadata["filename"] = document.metadata["source"]

In [16]:
import nest_asyncio
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

nest_asyncio.apply()

generator = TestsetGenerator.from_langchain(
    generator_llm=ChatOpenAI(model="gpt-4o-mini"),
    critic_llm=ChatOpenAI(model="gpt-4o-mini"),
    embeddings=OpenAIEmbeddings(),
)

testset = generator.generate_with_langchain_docs(
    documents[:10], # token不足で処理が終わらなかったので、documentを減らした
    test_size=4,
    distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25},
)

Generating: 100%|██████████| 4/4 [00:27<00:00,  6.75s/it]       


In [27]:
testset.to_pandas()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What should be described if a modification in ...,[Thanks for your contribution and we appreciat...,If a modification in a pull request breaks bac...,simple,[{'source': '.github/pull_request_template.md'...,True
1,What is the purpose of using model promotion i...,[�装\n\n请参考[快速入门文档](docs/zh_cn/get_started.md#i...,The answer to given question is not present in...,simple,"[{'source': 'README_zh-CN.md', 'file_path': 'R...",True
2,What can enhance a feature request's context?,[---\nname: Feature request\nabout: Suggest an...,A feature request's context can be enhanced by...,reasoning,[{'source': '.github/ISSUE_TEMPLATE/feature_re...,True
3,What community projects boost MMSegmentation i...,[# Projects\n\nThe OpenMMLab ecosystem can onl...,The community projects that boost MMSegmentati...,multi_context,"[{'source': 'projects/README.md', 'file_path':...",True


## LangSmith の Dataset の作成
* LangSmithに空のデータセットを作成する
* ローカルでデータセットを作成する

In [28]:
from langsmith import Client

dataset_name = "agent-book"

client = Client()

if client.has_dataset(dataset_name=dataset_name):
    client.delete_dataset(dataset_name=dataset_name)

dataset = client.create_dataset(dataset_name=dataset_name)

In [31]:
dataset

Dataset(name='agent-book', description=None, data_type=<DataType.kv: 'kv'>, id=UUID('1c14a6a6-5442-4190-b66c-d20a10e84a8d'), created_at=datetime.datetime(2025, 6, 27, 23, 18, 34, 719764, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2025, 6, 27, 23, 18, 34, 719764, tzinfo=datetime.timezone.utc), example_count=0, session_count=0, last_session_start_time=None, inputs_schema=None, outputs_schema=None)

## 合成テストデータの保存
* LangSmithに保存する形に変換
* LangSmithに送る

In [32]:
inputs = []
outputs = []
metadatas = []

for testset_record in testset.test_data:
    inputs.append(
        {
            "question": testset_record.question,
        }
    )
    outputs.append(
        {
            "contexts": testset_record.contexts,
            "ground_truth": testset_record.ground_truth,
        }
    )
    metadatas.append(
        {
            "source": testset_record.metadata[0]["source"],
            "evolution_type": testset_record.evolution_type,
        }
    )

In [33]:
client.create_examples(
    inputs=inputs,
    outputs=outputs,
    metadata=metadatas,
    dataset_id=dataset.id,
)

## LangSmith と Ragas を使ったオフライン評価の実装

### カスタム Evaluator の実装

In [3]:
from typing import Any

from langchain_core.embeddings import Embeddings
from langchain_core.language_models import BaseChatModel
from langsmith.schemas import Example, Run
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.llms import LangchainLLMWrapper
from ragas.metrics.base import Metric, MetricWithEmbeddings, MetricWithLLM


class RagasMetricEvaluator:
    def __init__(self, metric: Metric, llm: BaseChatModel, embeddings: Embeddings):
        self.metric = metric

        # LLMとEmbeddingsをMetricに設定
        if isinstance(self.metric, MetricWithLLM):
            self.metric.llm = LangchainLLMWrapper(llm)
        if isinstance(self.metric, MetricWithEmbeddings):
            self.metric.embeddings = LangchainEmbeddingsWrapper(embeddings)

    def evaluate(self, run: Run, example: Example) -> dict[str, Any]:
        context_strs = [doc.page_content for doc in run.outputs["contexts"]]

        # Ragasの評価メトリクスのscoreメソッドでスコアを算出
        score = self.metric.score(
            {
                "question": example.inputs["question"],
                "answer": run.outputs["answer"],
                "contexts": context_strs,
                "ground_truth": example.outputs["ground_truth"],
            },
        )
        return {"key": self.metric.name, "score": score}

In [4]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.metrics import answer_relevancy, context_precision

metrics = [context_precision, answer_relevancy]

llm = ChatOpenAI(model="gpt-4o", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

evaluators = [
    RagasMetricEvaluator(metric, llm, embeddings).evaluate
    for metric in metrics
]

### 推論の関数の実装

In [7]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(documents[:10], embeddings)

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

retriever = db.as_retriever()

chain = RunnableParallel(
    {
        "question": RunnablePassthrough(),
        "context": retriever,
    }
).assign(answer=prompt | model | StrOutputParser())

In [9]:
def predict(inputs: dict[str, Any]) -> dict[str, Any]:
    question = inputs["question"]
    output = chain.invoke(question)
    return {
        "contexts": output["context"],
        "answer": output["answer"],
    }

### オフライン評価の実装・実行

In [10]:
from langsmith.evaluation import evaluate

evaluate(
    predict,
    data="agent-book",
    evaluators=evaluators,
)

View the evaluation results for experiment: 'extraneous-limit-5' at:
https://smith.langchain.com/o/b6b53287-d527-4310-9a81-ec911aa50ffa/datasets/1c14a6a6-5442-4190-b66c-d20a10e84a8d/compare?selectedSessions=3c79937f-4d2b-414c-80d0-4d2cc0b4ebcb




0it [00:00, ?it/s]

Failed to multipart ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')trace=1b194075-9e78-4345-80e0-8423f5586884,id=1b194075-9e78-4345-80e0-8423f5586884; trace=2fbc3d47-6e49-44e4-a09d-ef37ea85961e,id=2fbc3d47-6e49-44e4-a09d-ef37ea85961e; trace=6a520ac7-0459-4cab-b4f6-659293b6dab2,id=6a520ac7-0459-4cab-b4f6-659293b6dab2; trace=6d7104dc-00b5-4af9-a837-57dabef945a1,id=6d7104dc-00b5-4af9-a837-57dabef945a1
Failed to multipart ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage lim

,inputs.question,outputs.contexts,outputs.answer,error,reference.contexts,reference.ground_truth,feedback.evaluate,execution_time,example_id,id
0,What should be described if a modification in ...,[page_content='Thanks for your contribution an...,If a modification in a pull request breaks bac...,None,[Thanks for your contribution and we appreciat...,If a modification in a pull request breaks bac...,None,3.898259,d96861ae-364a-4c9a-a2e2-47bea13c0dc7,6d7104dc-00b5-4af9-a837-57dabef945a1
1,What can enhance a feature request's context?,[page_content='---\nname: Feature request\nabo...,A feature request's context can be enhanced by...,None,[---\nname: Feature request\nabout: Suggest an...,A feature request's context can be enhanced by...,None,5.442625,7092ad57-5162-4a63-9379-e768b5a02c7d,2fbc3d47-6e49-44e4-a09d-ef37ea85961e
2,What is the purpose of using model promotion i...,[page_content='# Contributing to MMSegmentatio...,In the context of the MMSegmentation documenta...,None,[�装\n\n请参考[快速入门文档](docs/zh_cn/get_started.md#i...,The answer to given question is not present in...,None,6.118859,e4895194-4f97-4fa6-a23c-d7deb7cf4fb2,6a520ac7-0459-4cab-b4f6-659293b6dab2
3,What community projects boost MMSegmentation i...,[page_content='# Projects\n\nThe OpenMMLab eco...,MMSegmentation in OpenMMLab is boosted by seve...,None,[# Projects\n\nThe OpenMMLab ecosystem can onl...,The community projects that boost MMSegmentati...,None,12.182957,49c73a96-233a-4529-9e9a-b96b2aebaf38,1b194075-9e78-4345-80e0-8423f5586884


## LangSmith を使ったオンライン評価の実装

### フィードバックボタンを表示する関数の実装

In [42]:
!pip install ipywidgets 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [ipywidgets]


In [11]:
from uuid import UUID

import ipywidgets as widgets
from IPython.display import display
from langsmith import Client


def display_feedback_buttons(run_id: UUID) -> None:
    # GoodボタンとBadボタンを準備
    good_button = widgets.Button(
        description="Good",
        button_style="success",
        icon="thumbs-up",
    )
    bad_button = widgets.Button(
        description="Bad",
        button_style="danger",
        icon="thumbs-down",
    )

    # クリックされた際に実行される関数を定義
    def on_button_clicked(button: widgets.Button) -> None:
        if button == good_button:
            score = 1
        elif button == bad_button:
            score = 0
        else:
            raise ValueError(f"Unknown button: {button}")

        client = Client()
        client.create_feedback(run_id=run_id, key="thumbs", score=score)
        print("フィードバックを送信しました")

    # ボタンがクリックされた際にon_button_clicked関数を実行
    good_button.on_click(on_button_clicked)
    bad_button.on_click(on_button_clicked)

    # ボタンを表示
    display(good_button, bad_button)

In [ ]:
from langchain_core.tracers.context import collect_runs

# LangSmithのトレースのID(Run ID)を取得するため、collect_runs関数を使用
with collect_runs() as runs_cb:
    output = chain.invoke("mmsegmentationの概要を教えて")
    print(output["answer"])
    run_id = runs_cb.traced_runs[0].id

display_feedback_buttons(run_id)

Failed to multipart ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')trace=01a598a4-dca1-4d15-80ad-b9905215b53f,id=01a598a4-dca1-4d15-80ad-b9905215b53f; trace=01a598a4-dca1-4d15-80ad-b9905215b53f,id=4b1de308-5aca-484c-b323-50a30d32e66d; trace=01a598a4-dca1-4d15-80ad-b9905215b53f,id=57a3fc5d-4329-4489-a123-b1dd75b065a4; trace=01a598a4-dca1-4d15-80ad-b9905215b53f,id=2ac9b212-b618-4586-98bb-e6e1969f94db
Failed to multipart ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage lim

MMSegmentationは、PyTorchに基づいたオープンソースのセマンティックセグメンテーションツールボックスです。OpenMMLabプロジェクトの一部として開発されており、さまざまなセマンティックセグメンテーションアルゴリズムを統合し、統一されたベンチマークプラットフォームを提供します。

主な特徴には以下が含まれます：

- **モジュール化設計**: セグメンテーションフレームワークを異なるコンポーネントに分解し、ユーザーが異なるモジュールを組み合わせてカスタマイズしたモデルを簡単に構築できるようにしています。
- **豊富なアルゴリズムとモデルのサポート**: PSPNet、DeepLabV3、PSANet、DeepLabV3+など、さまざまな主流のセグメンテーションアルゴリズムをサポートしています。
- **高い効率性**: 他のセマンティックセグメンテーションライブラリと比較して、トレーニング速度が速いか、同等の性能を持っています。

MMSegmentationは、ユーザーが既存のアルゴリズムを再現し、自分自身の新しいモデルを開発するための柔軟で標準化されたツールキットを提供することを目指しています。


Button(button_style='success', description='Good', icon='thumbs-up', style=ButtonStyle())

Button(button_style='danger', description='Bad', icon='thumbs-down', style=ButtonStyle())

Failed to multipart ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')trace=01a598a4-dca1-4d15-80ad-b9905215b53f,id=dd4d4870-1ef0-460b-bb8a-2297f12d51c5; trace=01a598a4-dca1-4d15-80ad-b9905215b53f,id=01a598a4-dca1-4d15-80ad-b9905215b53f; trace=01a598a4-dca1-4d15-80ad-b9905215b53f,id=1e2fcdd8-dd0a-4344-bfc1-b594921299ab; trace=01a598a4-dca1-4d15-80ad-b9905215b53f,id=073619c2-66b5-4bc6-a72b-ca495cc0033b; trace=01a598a4-dca1-4d15-80ad-b9905215b53f,id=c013eddd-efe4-4121-8384-8272c24723b2; trace=01a598a4-dca1-4d15-80ad-b9905215b53f,id=1f9e671c-38ad-4dd0-9639-8ac699695f3c
